# COLLECTING IMDBIDS AND TITLES
---

Nhiệm vụ phần này: Từ danh sách các đường dẫn phim của trang web [Just Watch](https://www.justwatch.com/us?age_certifications=G,NC-17,PG,PG-13,R,UNRATED&release_year_from=2023&release_year_until=2024&sort_by=release_year), thực hiện cào thông tin **IMDbID** hoặc **Title** (trong đường dẫn nếu không có ID), **Release year**, **Genre**, **Age rating**, **Runtime** và lưu lại vào file **pre_data_part{x}.csv** với x tương ứng với số thứ tự của thành viên thực hiện cào.

## IMPORT

In [1]:
import time
from selenium import webdriver
from selenium.webdriver import ChromeOptions
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import re
import pandas as pd
import html
import os

## 1. CÀO THÔNG TIN IMDbID TRONG TRANG `JUST WATCH`

- Lấy danh sách các đường dẫn phim đã được thu thập trong phần trước từ file links.txt

In [2]:
links = []

# Đọc file
with open('../Data/links.txt', 'r') as f:
    links = [line.strip() for line in f]

- Phân chia phiên làm việc cho các thành viên chịu trách nhiệm phần thu thập dữ liệu. Thông tin của mỗi thành viên sẽ bao gồm: 

    - **part**: phần nội dung mà thành viên đó sẽ chịu trách nhiệm

    - **start**: index bắt đầu của đường dẫn từ file links.txt

    - **end**: index kết thúc của đường dẫn từ file links.txt

In [3]:
# Nhiệm vụ của các thành viên
member_part = {
    "Trinh": {
        "part": 1,
        "start": 0,
        "end": 628
    },

    "Thi": {
        "part": 2,
        "start": 629,
        "end": 1258
    },

    "MinhUyen": {
        "part": 3,
        "start": 1259,
        "end": 1887
    } 
}

- Mỗi thành viên tiến hành thu thập dữ liệu dựa trên phần nhiệm vụ đã được phân công

- Việc thu thập sẽ được thực hiện độc lập giữa 3 thành viên

- Mỗi thành viên điền tên mình để thực hiện nhiệm vụ được giao

In [4]:
# Điền tên để tiến hành cào
name = "MinhUyen"
info = member_part[name]

part = info["part"]
start = info["start"]
end = info["end"]

- Tiến hành thu thập **IMDbID** và **Title**

In [5]:
# Lấy ID và title của phim dựa vào link cho trước
def collect_imdbID_title(links, start, end):
    # Thiết lập tùy chọn cho trình duyệt Chrome
    option = webdriver.ChromeOptions()

    prefs = {
        "profile.managed_default_content_settings.images": 2,
        "profile.managed_default_content_settings.plugins": 2,
        "profile.managed_default_content_settings.media": 2
    }
    option.add_experimental_option("prefs", prefs)

    # Khởi tạo trình duyệt Chrome với tùy chọn trên
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=option)
    
    # Khởi tạo các danh sách lưu trữ thông tin cần thu thập
    imdbIds_titles = []
    years = []
    genres = []
    age_ratings = []
    runtimes = []

    # Duyệt qua từng URL trong khoảng chỉ định (start -> end)
    for url in links[start:end + 1]:
        print(f"[LOG]: parse imdbid of url {url}")
        driver.get(url)
        time.sleep(4)
        page_source = driver.page_source    # Lấy mã nguồn HTML của trang
        page_source = html.unescape(page_source)    # Giải mã các ký tự HTML đặc biệt

        # Tìm IMDb ID từ mã nguồn trang web
        imdb_id_matches = re.findall(r'"imdbId":"(tt\d+)"', page_source)
        if imdb_id_matches:
            imdb_id = imdb_id_matches[-1]   # Lấy IMDb ID cuối cùng (nếu có nhiều kết quả)
            item = imdb_id
        else:
            # Nếu không tìm thấy IMDb ID, lấy tiêu đề phim từ thẻ meta
            movie_title_meta = driver.find_element(By.XPATH, "//meta[@property='og:title']")
            if movie_title_meta:
                movie_title = movie_title_meta.get_attribute("content").split(" streaming")[0]
                item = movie_title
            else:
                item = "Null"

        # Tìm thể loại phim (Genres) từ mã nguồn trang web
        genre_match = re.search(r'Genres<\/h3><div class="detail-infos__value".*?<span data-v.*?>(.*?)</span>', page_source, re.DOTALL)
        genre = genre_match.group(1) if genre_match else "Null"
        if genre.startswith("<a"): # xử lý trường hợp đặc biệt
            genre = "Mystery & Thriller"

        # Tìm thời lượng phim (Runtime)
        runtime_match = re.search(r'Runtime<\/h3><div class="detail-infos__value"[^>]*>(.*?)<\/div>', page_source)
        runtime = runtime_match.group(1) if runtime_match else "Null"

        # Tìm độ tuổi giới hạn (Age rating)
        age_rating_match = re.search(r'Age rating<\/h3><div class="detail-infos__value"[^>]*>(.*?)<\/div>', page_source)
        age_rating = age_rating_match.group(1) if age_rating_match else "Null"

        # Tìm năm phát hành phim (Release year)
        release_year_match = re.search(r'class="release-year".*?\((\d{4})\)', page_source)
        release_year = release_year_match.group(1) if release_year_match else "Null"

        # In ra kết quả
        print(f"{item} - {release_year} - {age_rating} - {runtime} - {genre}")

        # Cập nhật thông tin đã thu thập được
        imdbIds_titles.append(item)
        years.append(release_year)
        runtimes.append(runtime)
        age_ratings.append(age_rating)
        genres.append(genre)

    # Đóng trình duyệt khi hoàn tất
    driver.quit()

    # Lưu dữ liệu
    df = pd.DataFrame({
        "Id/Title": imdbIds_titles,     # Cột chứa IMDb ID hoặc tiêu đề phim
        "Release year": years,          # Cột chứa năm phát hành
        "Age rating": age_ratings,      # Cột chứa phân loại độ tuổi
        "Runtime": runtimes,            # Cột chứa thời lượng phim
        "Genre": genres                 # Cột chứa thể loại phim
    })

    return df

In [43]:
# In kết quả
pre_data = collect_imdbID_title(links, start=start, end=end)
pre_data

[LOG]: parse imdbid of url https://www.justwatch.com/us/movie/in-a-silent-world
tt5217950 - 2023 - G - 1h 29min - Documentary
[LOG]: parse imdbid of url https://www.justwatch.com/us/movie/vikings-battle-of-heirs
tt21923666 - 2023 - PG - 1h 22min - Mystery & Thriller
[LOG]: parse imdbid of url https://www.justwatch.com/us/movie/mistletoe-connection
tt28225059 - 2023 - G - 1h 30min - Comedy, Romance, Drama
[LOG]: parse imdbid of url https://www.justwatch.com/us/movie/disney-intertwined-live
tt27512578 - 2023 - PG - 1h 8min - Music & Musical, Comedy, Drama, Fantasy
[LOG]: parse imdbid of url https://www.justwatch.com/us/movie/thabo-the-rhino-adventure
tt22755734 - 2023 - PG - 1h 37min - Kids & Family, Action & Adventure, Mystery & Thriller
[LOG]: parse imdbid of url https://www.justwatch.com/us/movie/the-modelizer
tt13482930 - 2023 - R - 1h 36min - Romance, Comedy
[LOG]: parse imdbid of url https://www.justwatch.com/us/movie/trapped-in-the-farmhouse
tt27144976 - 2023 - PG-13 - 1h 30min - 

,Id/Title,Release year,Age rating,Runtime,Genre
0,tt5217950,2023,G,1h 29min,Documentary
1,tt21923666,2023,PG,1h 22min,Mystery & Thriller
2,tt28225059,2023,G,1h 30min,"Comedy, Romance, Drama"
3,tt27512578,2023,PG,1h 8min,"Music & Musical, Comedy, Drama, Fantasy"
4,tt22755734,2023,PG,1h 37min,"Kids & Family, Action & Adventure, Mystery & T..."
...,...,...,...,...,...
82,tt21156390,2023,R,1h 56min,Drama
83,tt29931745,2023,G,1h 24min,"Comedy, Romance, Drama"
84,tt9288740,2023,R,1h 34min,"Horror, Mystery & Thriller"
85,tt24548912,2023,PG,1h 23min,"Animation, Action & Adventure, Fantasy"


## 2. LƯU FILE

Lưu thông tin vừa thu thập được vào file csv

In [40]:
file_name = f"../Data/pre_data_part{part}.csv"

if os.path.isfile(file_name):
    pre_data.to_csv(file_name, mode='a', header=False, index=False)
else:
    pre_data.to_csv(file_name, mode='w', header=True, index=False)